# Arc Results

In [ ]:
%matplotlib inline
from arc_maps import get_data
from smif.data_layer import Results
import seaborn as sbn
import matplotlib.pyplot as plt
import numpy as np
results = Results({'interface': 'local_csv', 'dir': '../'})

sbn.set_context('talk', rc={"lines.linewidth": 2.5})
sbn.set_palette('Paired')

In [ ]:
filepath = '../credential.ini'
arc_lads = get_data.get_lads(filepath)

# View the LADs

In [ ]:
def highlight(data, id_col='geo_label'):
    
    def group_hlads(index):
        if index in {'Bedford', 'Cambridge', 'Milton Keynes', 'Oxford', 
                     'Aylesbury Vale', 'Central Bedfordshire', 'Huntingdonshire', 'South Cambridgeshire'}:
            return index
        else:
            return 'Other'
    
    data['summary_lads'] = data[id_col].apply(group_hlads) 
    
    return data

def plot_highlighted_lads(arc_lads):
    
    arc_lads = highlight(arc_lads.reset_index())
    
    # Plot the LADs
    ax = arc_lads.plot(column='summary_lads', edgecolor='grey')
    
    # Add the labels
    centroids = [(float(x.x), float(x.y)) for x in list(arc_lads.centroid)]
    for label, coord in zip(arc_lads['summary_lads'].values, centroids):
        if label != 'Other':
            plt.annotate(label, coord, ha='center', va='center',
                         color='black', weight='bold')
    ax.set_axis_off()
    return ax
    

with sbn.plotting_context('paper'):
    sbn.set(rc={'figure.figsize': (11.7, 8.27)})
    ax = plot_highlighted_lads(arc_lads)
    ax.get_figure().savefig("lads_highlighted.png")

In [ ]:
highlight(arc_lads.reset_index())

In [ ]:
results.list_model_runs()

In [ ]:
results.available_results('arc_ets__baseline')

In [ ]:
results.list_scenario_outputs('socio-economic')
dwellings = results.read_scenario_data(
    'socio-economic', 'arc_baseline', 'dwellings', [2015, 2020, 2030, 2050])
# sbn.relplot(x='timestep', y='gva_per_sector', hue='sectors', kind='line', data=gva.reset_index())

# Plotting Arc scenario data

The outputs of the socio economic scenarios include:

In [ ]:
results.list_scenario_outputs('socio-economic')

In [ ]:
results.available_results('arc_et__baseline')

In [ ]:
def get_scenario_data(scenario_output):

    arc_lads = get_data.get_lads(filepath)

    scenarios = ['arc_unplanned', 'arc_new-cities', 'arc_expansion']
    result = []

    for scenario in scenarios:

        result.append(results.read_scenario_data(
        'socio-economic', scenario, scenario_output, [2015, 2020, 2030, 2050]).set_index(
        ['lad_uk_2016', 'timestep']))

    # Make the initial df from the baseline data
    baseline_data = results.read_scenario_data(
        'socio-economic', 'arc_baseline', scenario_output, [2015, 2020, 2030, 2050]).set_index(
        ['lad_uk_2016', 'timestep']).rename(columns={scenario_output: 'baseline'})
    
    for name, df in zip(scenarios, result):

        df = df.reset_index().set_index(['lad_uk_2016', 'timestep'])
        suffix = "_{}".format(name.split("_")[1])
        baseline_data[suffix[1:]] = df[scenario_output]

    arc_data = arc_lads.join(
        baseline_data.reset_index().set_index(
        ['lad_uk_2016'])).drop(
        columns=['gid']).rename(
        columns={scenario_output: 'baseline'})
    
    return arc_data


In [ ]:
def reshape(df):
    
    return df.melt(id_vars=['geo_label', 'timestep', 'geometry'], 
                  value_vars=['baseline',  'unplanned', 'new-cities', 'expansion'],
                  var_name='scenario')

def plot_total_scenario_data(df, y_label=None):
    data = reshape(df).groupby(by=['scenario', 'timestep']).sum().reset_index()
    fig = sbn.relplot(x='timestep', y='value', hue='scenario', data=data, kind='line',
                      height=4)
    fig.set_axis_labels('year', y_label)
    return fig
    
def plot_scenario_data(df):
#     sbn.set_palette("Paired")
    data = reshape(df)
      
    return sbn.relplot(x='timestep', y='value', hue='geo_label', data=data, col='scenario', kind='line',
                      col_wrap=2)
    
def plot_scenario_data_highlight(df, y_label=None):
    
    data = reshape(df)
    data = highlight(data)   
    fig = sbn.relplot(x='timestep', y='value', hue='summary_lads', 
                      data=data, col='scenario', kind='line', col_wrap=2)
    fig.set_axis_labels('year', y_label)
    return fig

def plot_bar_highlight(df, y_label=None):
    
    data = reshape(df)
    data = highlight(data)
    fig = sbn.catplot(x='timestep', y='value', hue='summary_lads', data=data, col='scenario', 
                      kind='bar', col_wrap=2)
    fig.set_axis_labels('year', y_label)
    return fig
   

In [ ]:
arc_dwellings = get_scenario_data('dwellings')
arc_floor = get_scenario_data('floor_area')
arc_population = get_scenario_data('population')
arc_jobs = get_scenario_data('employment')


In [ ]:
# arc_dwellings.head()
plot_scenario_data(arc_dwellings)

In [ ]:
fig = plot_scenario_data_highlight(arc_dwellings, 'dwellings')
fig.savefig("dwellings_highlight.png")

In [ ]:
filtered = arc_floor[arc_floor['residential_or_non']=='residential']
fig = plot_scenario_data_highlight(filtered, 'floor area')
fig.savefig("floor_area_highlight.png")

In [ ]:
plot_bar_highlight(arc_dwellings, 'dwellings')

In [ ]:
fig = plot_scenario_data_highlight(arc_population, 'people')
fig.savefig("population_highlight.png")

In [ ]:
arc_gva = get_scenario_data('gva')

In [ ]:
arc_gva.head()

In [ ]:
plot_scenario_data_highlight(arc_gva)

In [ ]:
plot_total_scenario_data(arc_dwellings, 'dwellings').savefig("total_dwellings.png")

In [ ]:
plot_total_scenario_data(arc_gva, 'gva').savefig("total_gva.png")

In [ ]:
plot_total_scenario_data(arc_population, 'people').savefig("total_population.png")

In [ ]:
plot_total_scenario_data(arc_jobs, 'jobs').savefig("total_jobs.png")

In [ ]:
arc_jobs.head(10)

In [ ]:
fig = plot_scenario_data_highlight(arc_jobs)
fig.savefig("jobs_highlight.png")

In [ ]:
fig = plot_scenario_data_highlight(arc_gva)
fig.savefig("gva_highlight.png")

# Plot difference spatially

In [ ]:
def plot_difference_spatially(df, arc_lads, remove_2015=True):

    melted_dwellings = reshape(df).drop(columns='geometry')
    
    def log(series):
        return series.apply(lambda x: np.log(x))
    
    melted_dwellings['norm_value'] = log(melted_dwellings['value'])
    
    baseline = melted_dwellings[melted_dwellings['scenario']=='baseline'].drop(columns='scenario')
    non_baseline = melted_dwellings[~(melted_dwellings['scenario']=='baseline')]
    
    diff = non_baseline.set_index(['scenario', 'geo_label', 'timestep']).sub(
                baseline.set_index(['geo_label', 'timestep'])).reset_index()
    diff = highlight(diff)  
    
    diff_spatial = arc_lads.reset_index().set_index('geo_label').join(
                       diff.set_index('geo_label'), rsuffix='bla')
    
    cmap = sbn.diverging_palette(h_neg=220, h_pos=10, s=74, l=50, sep=10, n=9, as_cmap=True)
    
    def plot_diff_map(data, *args, **kwargs):
        cax = plt.gca()
        ax = data.plot(column='norm_value', ax=cax, cmap=cmap, 
                       vmin=-diff_spatial['norm_value'].max(), vmax=diff_spatial['norm_value'].max())
        ax.set_axis_off()
        return ax

    # Remove baseline (no difference across scenarios)
    if remove_2015:
        diff_spatial = diff_spatial[~(diff_spatial['timestep'] == 2015)]

    with sbn.plotting_context('paper'):
        g = sbn.FacetGrid(data=diff_spatial, row='scenario', col='timestep', 
                          height=4, sharex=True, sharey=True)
        g.map_dataframe(plot_diff_map)
        
    return g

In [ ]:
plot_difference_spatially(arc_dwellings, arc_lads).savefig("dwellings_difference.png")

In [ ]:
plot_difference_spatially(arc_population, arc_lads).savefig('population_difference.png')

In [ ]:
plot_difference_spatially(arc_jobs, arc_lads, remove_2015=False).savefig('jobs_difference.png')

In [ ]:
plot_difference_spatially(arc_gva, arc_lads, remove_2015=False).savefig('gva_difference.png')

# Baseline Results

In [ ]:
results.available_results('arc_ets__baseline')

In [ ]:
results.list_sector_models('arc_ets__baseline')

In [ ]:
results.list_outputs('aggregate_energy_constrained')

In [ ]:
model_run_names = ['arc_ets__baseline', 'arc_ets__expansion']
model_names = ['aggregate_energy_constrained']
output_names = ['elecload']
timesteps=[2015, 2030, 2050]
elecload = arc_lads.join(
    results.read_results(
        model_run_names, model_names, output_names, timesteps).set_index('lad_uk_2016').drop(columns=['decision']))

In [ ]:
arc_elecload = elecload.reset_index().melt(
    id_vars=['index', 'geo_label', 'geometry', 'gid', 'timestep', 'hourly', 'model_run'], 
    value_vars='elecload')

In [ ]:
arc_elecload.head()

In [ ]:
data = arc_elecload.drop(columns=['geometry', 'gid'])
data = data.groupby(by=['model_run', 'timestep', 'hourly']).sum().reset_index()

sbn.relplot(x='hourly', y='value', row='model_run', col='timestep', data=data, kind='line')

# Visualise sos as a graph

In [ ]:
import networkx as nx

In [ ]:
def visualise_sos(store, model_run):
    """Write out a graphml file showing directed links between scenarios, models and adaptors
    """
    config = store.read_model_run(model_run)
    sos = store.read_sos_model(config['sos_model'])
    models = sos['sector_models']
    scenarios = sos['scenarios']
    nodes = models + scenarios
    G = nx.DiGraph(model_run='arc_ets__expansion', sos_model=sos['name'])
    for x in nodes:
        G.add_node(x, name=x) 
    for dep in sos['model_dependencies'] + sos['scenario_dependencies']:
        G.add_edge(dep['source'], dep['sink'])
    nx.write_graphml_lxml(G, "{}.graphml".format(model_run))

In [ ]:
store = results._store
model_run = 'arc_et__expansion'
visualise_sos(store, model_run)

# Debugging expansion scenario

In [ ]:
model_run = ['arc_ets__expansion']
model_names = ['energy_demand_constrained']
output_names = ['industry_gas_boiler_gas']
results.available_results(model_run[0])['sector_models'][model_names[0]]['outputs'].keys()

In [ ]:
data = results.read_results(model_run, model_names, output_names)

In [ ]:
# Show all entries where there is missing data
data[data['industry_gas_boiler_gas'].isnull()==True]

# Investigate GVA sectoral and total 

In [ ]:
def get_scenario_data(variable, timesteps):
    data = results.read_scenario_data('socio-economic', 'arc_baseline', variable, timesteps).rename(
        columns={variable: 'baseline'})

    data['unplanned'] = results.read_scenario_data(
        'socio-economic', 'arc_unplanned', variable, timesteps)[variable]
    data['expansion'] = results.read_scenario_data(
        'socio-economic', 'arc_expansion', variable, timesteps)[variable]
    data['new-cities'] = results.read_scenario_data(
        'socio-economic', 'arc_new-cities', variable, timesteps)[variable]
    
    return data

head = get_scenario_data('gva_per_head', [2015, 2030, 2050])
head.sample(5)

In [ ]:
gva = get_scenario_data('gva', [2015, 2020, 2030, 2050])
gva.sample(5)

In [ ]:
sector = get_scenario_data('gva_per_sector', [2015, 2020, 2030, 2050])
sector.tail(5)

In [ ]:
sbn.relplot(data=sector.melt(id_vars=['lad_uk_2016', 'timestep', 'sectors'], 
                             var_name='scenario').groupby(by=['scenario', 'timestep']).sum().reset_index(), 
            x='timestep', y='value', 
            col='scenario', kind='line')

In [ ]:
gva_melted = gva.melt(id_vars=['lad_uk_2016', 'timestep'], var_name='scenario')

In [ ]:
summed = sector.melt(
    id_vars=['lad_uk_2016', 'timestep', 'sectors'], 
    var_name='scenario').groupby(
    by=['scenario', 'timestep']).sum().drop(columns='sectors')

In [ ]:
summed

In [ ]:
gva_melted.groupby(by=['scenario', 'timestep']).sum()